In [1]:
#### Descobrir o portfolio ###

In [6]:
import investpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
## Ativos e Periodos

ativos = ['ITUB4', 'PETR4', 'BBDC4', 'ABEV3', 'MGLU3', 'WEGE3', 'BBAS3', \
        'VALE3', 'LAME4', 'VVAR3', 'BBDC3', 'BPAC11', 'BBSE3', 'ELET6']

fromdate = '01/01/2020'
todate = '01/08/2022'

In [8]:
## Baixa os ativos para um DF
df_precos = pd.DataFrame()

for acao in ativos:
    
    try:
        p = pd.Series(investpy.get_stock_historical_data(stock=acao,
                   country='brazil',
                   from_date= fromdate,
                   to_date= todate)['Close'],name=acao)
    
        df_precos = pd.concat([df_precos,p], axis = 1)
    except:
        pass

df_precos.dropna(how='any', inplace=True)

In [ ]:
### Cria um portfolio aleatório

def gera_porfolio(ativos, duracao, seed = 0):

    if seed